# Annual Energy Savings from Recycled Materials in Singapore

## Project Goals
The goal of this project is to analyze the total garbage collection and recycling rate in Singapore, and to determine the amount of energy saved from recycling.

In this analysis, we will answer questions such as:
1. How much energy was saved per year? In which year was this amount the highest? The lowest? 
2. What is the trend for recycled energy savings in Singapore from 2003 to 2022?
3. What is the greatest source of recycled energy savings in 2022 and how has this changed over time?

For more information about how recycling can save energy, please refer here: https://greentumble.com/how-does-recycling-save-energy

## Data
- Recycled energy data for 2003 to 2016 a csv file is taken from the reference for this project, [kingabzpro](https://github.com/kingabzpro/Annual-Recycled-Energy-Saved-in-Singapore/tree/main/Data)
- Recycled energy data for 2017 to 2021 is taken from the Waste and Recycling Statistics [document](https://www.nea.gov.sg/docs/default-source/default-document-library/waste-and-recycling-statistics-2017-to-2021.pdf) on the NEA website. The data has been extracted to an Excel file.
- Recycled energy data for 2022 is taken from the [Waste Statistics and Overall Recycling NEA webpage](https://www.nea.gov.sg/our-services/waste-management/waste-statistics-and-overall-recycling)

**Data Dictionary**

|Variable|Description|
|-----|-----|

## Table of Contents
1. Data Acquisition
2. Data Cleaning and Pre-processing
3. Data Exploration and Visualization
4. Conclusions

***

## 1. Data Acquisition

#### Import Libraries

In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

import sqlite3

#### 2003-2016: Import Data from `.csv`

In [2]:
# 2003 - 2016
df_03to16 = pd.read_csv('data/waste-and-recycling-statistics-2003-to-2016.csv')

In [3]:
df_03to16

,waste_type,waste_disposed_of_tonne,total_waste_recycled_tonne,total_waste_generated_tonne,recycling_rate,year
0,Food,679900,111100.0,791000,0.14,2016
1,Paper/Cardboard,576000,607100.0,1183100,0.51,2016
2,Plastics,762700,59500.0,822200,0.07,2016
3,C&D,9700,1585700.0,1595400,0.99,2016
4,Horticultural waste,111500,209000.0,320500,0.65,2016
...,...,...,...,...,...,...
220,Ash and sludge,214800,28600.0,243400,0.12,2017
221,Plastic,763400,51800.0,815200,0.06,2017
222,Textile/Leather,141200,9600.0,150800,0.06,2017
223,"Others (stones, ceramic, rubber, etc.)",319300,7100.0,326400,0.02,2017


#### 2017-2021: Import Data from `.xlsx`

In [4]:
# 2017-2021
sheets = ['2017', '2018', '2019', '2020', '2021']

df_17to21_list = []
for sheet in sheets:
    df = pd.read_excel('data/waste-and-recycling-statistics-2017-to-2021.xlsx', sheet_name=sheet)
    df = df.rename(columns=df.iloc[0]).loc[1:]
    df['year'] = sheet
    df_17to21_list.append(df)
    
df_17to21 = pd.concat(df_17to21_list, axis=0)

In [5]:
df_17to21

,Waste Type,Total Generated\n('000 tonnes),Total Recycled\n('000 tonnes),Recycling Rate,Total Disposed\n('000 tonnes),year
1,C&D,"1,609","1,600",99%,9,2017
2,Ferrous metal,"1,379","1,371",99%,8,2017
3,Paper/Cardboard,"1,145",569,50%,576,2017
4,Plastics,815,52,6%,763,2017
5,Food,810,133,16%,677,2017
...,...,...,...,...,...,...
11,Non-ferrous metal,88,87,98%,1,2021
12,Glass,74,9,13%,65,2021
13,Scrap tyres,27,26,95%,1,2021
14,"Others (stones, ceramics, etc.)",233,18,8%,214,2021


#### 2022: Scrape Data with BeautifulSoup

In [6]:
#2022
url = 'https://www.nea.gov.sg/our-services/waste-management/waste-statistics-and-overall-recycling'
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find('table')
    data = [(cell.text for cell in row.find_all('td')) for row in table.find_all('tr')]

df_22 = pd.DataFrame(data)

In [7]:
df_22

,0,1,2,3,4
0,None,None,None,None,None
1,Ferrous metal,"1,338","1,331",99%,7
2,Paper/Cardboard,"1,064",394,37%,671
3,Construction & Demolition,"1,424","1,419",99%,5
4,Plastics,"1,001",57,6%,944
5,Food,813,146,18%,667
6,Horticultural,221,188,85%,32
7,Wood,419,298,71%,121
8,Ash & sludge,241,27,11%,213
9,Textile/Leather,254,5,2%,249


***

## 2. Data Cleaning and Pre-processing

### Cleaning `df_03to16`

In [8]:
# check duplicates

In [9]:
# check missing values

### Cleaning `df_17to22`

### Cleaning `df_22`

### Doing it altogether

***

## 3. Data Exploration and Visualization

In [ ]:
# create new database
conn=sqlite3.connect('mydb.db')

In [ ]:
# use pandas `.to_sql` to create a table 'recycling' from dataframe df
df.to_sql(name='recycling', con=conn, if_exists='replace', index=False)
conn.comit()

In [ ]:
# connect to database
%load_ext sql
%sql sqlite:///mydb.db

In [ ]:
# start querying!

In [10]:
# recycling rate of individual waste types per year
%%sql

In [11]:
# total energy saved per year
%%sql

***

## 4. Conclusions

***